In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

import sys
sys.path.append("../scripts")

import preprocessing as dp
import data_extraction as de
import exploratory_analysis as eda
import hypothesis_testing as ab

In [2]:
df = de.load_data('../data/csv.csv')

d:\Personal Files\KIAM Notebooks\KIAM-Week3\notebooks\../scripts\data_extraction.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Data Loaded Successfully


In [3]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,3/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0
1,145249,12827,5/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0
2,145249,12827,7/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0
3,145255,12827,5/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0
4,145255,12827,7/1/2015 0:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               104888 non-null   object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

In [5]:
# duplicated values
df.duplicated().sum()

np.int64(0)

In [6]:
# Drop Duplicates
df.drop_duplicates(inplace=True)

In [7]:
## Initialize the Hypothesis Testing class

testing = ab.HypothesisTesting(df)


## **Hypothesis Testing**

#### ***1. There are no risk differences across provinces***

In this Hypothesis we test if there is any satatistical evidence to support the null hypothesis which is : There are no risk differences across provinces.

In [8]:
testing.risk_difference_across_provinces()

Test : ANOVA
T-test F-statistic: 2.161069458248007
T-test p-value: 0.027241688082460597
Reject the null hypothesis: There are significant risk differences across provinces.


### ***Result***

The statistical test suggests we reject the null Hypothesis. There is a significant statistical evidence that suggest there is a risk difference across provinces.

### ***2. There are no risk differences between zip codes***

In this hypothesis we test if there is any significant statistical evidence that support the null hypothesis : There are no risk differences across zip codes.

In [9]:
testing.risk_difference_across_zip_codes()

Test: T-test
T-test T-statistic: -0.4959072629524582
T-test p-value: 0.619959950628635
Fail to reject the null hypothesis: No significant risk differences between zip codes.


### ***Result***

The test suggests to accept the null hypothesis : There are no signficant risk differences between zip codes

### ***3. There are no significant margin (profit) difference between zip codes***

In this hypothesis we test if there is any significant statistical evidence that support the null hypothesis : There are no significant margin (profit) difference between zip codes.

In [10]:
testing.profit_difference_across_zip_codes()

Test: T-test
T-test T-statistic: -0.400231005905576
T-test p-value: 0.6889864649725321
Fail to reject the null hypothesis: No significant Margin(Profit) differences between zip codes.


### ***Result***

We fail to reject the null hypohtesis. There are no significant Profit differences across Zip codes

### ***4. There are not significant risk difference between Women and Men***

In this hypothesis we test if there is any significant statistical evidence that support the null hypothesis : There are no significant risk difference between Women and Men.

In [11]:
testing.risk_difference_across_gender()

Test: T-test
T-test T-statistic: 0.6959441469250746
T-test p-value: 0.4864670714626016
Fail to reject the null hypothesis: No significant risk differences between women and men.


### ***Result***

We fail to reject the null hypothesis. There are no significant risk differences between Genders.